<a href="https://colab.research.google.com/github/vikniksor/DataScience/blob/main/credit_defaults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import pickle

In [ ]:
df = pd.read_csv("../data/dzSVM.csv")
# # Исследуем данные
# Сколько классов? Объектов?
n_samples, n_features = df.shape
print(f"Количество наблюдений: {n_samples}")
print(f"Количество атрибутов: {n_features}")

In [ ]:
pd.set_option("display.max_columns", None)
print("Первые пять наблюдений: ")
print(df.head())
# Есть ли странности: столетние кредиты, возраст заемщика больше ста и т.д?
print("Статистика по данным: ")
print(df.describe(include="all"))

In [ ]:
plt.hist(df[~np.isnan(df["CLAGE"])]["CLAGE"])
plt.show()

In [ ]:
clage700orMore = df.CLAGE[df.CLAGE >= 700].count()
print("%f процентов значений CLAGE >= 700 (кредит старше 58 лет), всего %i наблюдений"
      % ((clage700orMore/df.CLAGE.count())*100, clage700orMore))
# Уберем эти наблюдения
df.drop(df[df.CLAGE >= 700].index, inplace=True)

In [ ]:
# Просмотрим количество пустых значений
print("Количество пустых значений: ")
print(df.isnull().sum())
# Заполним пустые значения медианным значением каждого столбца
df = df.fillna(df.median())
print("Количество пустых значений после изменений: ")
print(df.isnull().sum())
# Остались категориальные атрибуты. Заполним их самым частым значением
df = df.fillna(df.mode().iloc[0])
print("Количество пустых значений после заполнения категориальных переменных: ")
print(df.isnull().sum())

In [ ]:
print("Чистые данные: ")
print(df.describe(include="all"))

In [ ]:
# Проверим насколько сбалансированны классы
df["BAD"].value_counts().plot(kind="bar")
plt.title("Bad")
plt.show()

In [ ]:
print("%f процентов заемщиков не выплатили кредит"
      % ((df.BAD[df.BAD == 1].count()/df.BAD.count())*100))

In [ ]:
# # Нормализую данные: привожу в вид от 0 до 1
numeric_features = df.select_dtypes(include=[np.number])
print("Численные атрибуты: ", numeric_features.columns.values)
print("До нормализации: ")
print(numeric_features.describe())

In [ ]:
numeric_features_scaled = ((numeric_features - numeric_features.min()) /
                           (numeric_features.max() - numeric_features.min()))
print("После нормализации:")
print(numeric_features_scaled.describe())

In [ ]:
df[numeric_features.columns.values] = numeric_features_scaled[numeric_features.columns.values]

In [ ]:
print("Чистые и нормализованные данные:")
print(df.describe(include="all"))

In [ ]:
# Заменим категориальные атрибуты(JOB and REASON) на фиктивные значение 0 или 1
df = pd.get_dummies(df, drop_first=True)
print("Первые 5 наблюдений после замены на фиктивные значения: ")
print(df.head())
print("Количество наблюдений и атрибутов после замены: ", df.shape)
print("Чистые и нормализованные данные c фиктивными значениями вместо категориальных:")
print(df.describe(include="all"))

In [ ]:
# Проверим есть ли корреляция между атрибутами. Сохраним корреляционную матрицу в Excel-файле:
corr = df.corr()
corr.to_excel("../data/Correlations.xlsx")

In [ ]:
# Просмотр корреляции между значениями не в Excel:
triangle = corr.abs().where(np.tril(np.ones(corr.shape), k=-1).astype(np.bool))
print("Самая сильная корреляция:")
print(triangle.stack().sort_values(ascending=False)[:7])

In [ ]:
# # Разделим на тренировочную и тестовую выборки
# Сперва разделим данные на X (все атрибуты) и y (колонка BAD: то, что надо предсказать):
y = df.BAD
X = df.drop("BAD", axis=1)

In [ ]:
# Теперь разделим данные на две части, на 70% данных будем обучать модель, 30% отложим для тестирования:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
# Параметр stratify гарантирует, что пропорции классов (20% невыплат) будут одинаковыми в тестовой и в учебной выборках)

In [ ]:
# # Определяем оптимальные гиперпараметры
# Создание классификатора:
clsf = SVC(class_weight="balanced", kernel="rbf")
# Выбор гиперпараметров: пробуем C от 0,5 до 5000 и gamma от 0,01 до 1:
param_distributions = {"C": sp.stats.uniform(0.5, 5000), "gamma": sp.stats.uniform(0.01, 1)}

In [ ]:
# # Начинаем обучение модели:
# Пробуем 40 разных сочетаний гиперпараметров, тестируем каждое сочетание 4 раза (перекрёстная проверка),
# оцениваем по количеству правильно классифицированных наблюдений в обоих классах:
random_search = RandomizedSearchCV(clsf, param_distributions=param_distributions,
                                   n_iter=40, cv=4, scoring="balanced_accuracy", n_jobs=-1)
random_search.fit(X_train, y_train)
# Сохранаяем оптимальную модель и смотрми на ее параметры:
model = random_search.best_estimator_
print("Оптимальные параметры: %s, оценка на учебных данных: %0.2f"
      % (random_search.best_params_

In [ ]:
# Сохраним модель:
filename = 'svc_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# # Оценка модели
model = pickle.load(open(filename, 'rb'))
y_pred = model.predict(X_test)

print("Результат на тестовых данных: %f" %
      (100*metrics.balanced_accuracy_score(y_test, y_pred)))

In [ ]:
# Посмотрим на конкретное количество наблюдений,
# записанных классификатором в тот или иной класс, для этого посчитаем матрицу неточностей:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

In [ ]:
print("Матрица неточностей:")
print(cnf_matrix)

In [ ]:
# Для наглядности можно показать матрицу на графике:
sb.heatmap(cnf_matrix, annot=True, cmap="Blues", fmt="g",
           xticklabels=["Выплата", "Невыплата"], yticklabels=["Выплата", "Невыплата"])
plt.ylabel("Реальное значение")
plt.xlabel("Предсказанное значение")
plt.show()